In [26]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
import torch.nn.init as init
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils_training import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [ ]:
N_train = 1000
N_bound = 100

# x,t
la = np.array([1,1])
lb = np.array([-1,0])

traindata = lb+(la-lb)*lhs(2,N_train)
x_inside = traindata[:,0:1]
t_inside = traindata[:,1:2]

x_inside = numpy_to_tensor(x_inside, var_name="x_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
t_inside = numpy_to_tensor(t_inside, var_name="t_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

x_bound = lb[0]+(la[0]-lb[0])*lhs(1,N_bound)
t_bound = lb[1]+(la[1]-lb[1])*lhs(1,N_bound)

x_bound = numpy_to_tensor(x_bound, var_name="x_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)
t_bound = numpy_to_tensor(t_bound, var_name="t_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

In [ ]:
from scipy.io import loadmat

data = loadmat("usol_D_0.001_k_5.mat")

t_exact = data["t"].reshape(-1,1)

x_exact = data["x"].reshape(-1,1)

Exact_u = data["u"]

print('t_exact:',t_exact.shape)
print('x_exact:',x_exact.shape)
print('Exact_u:',Exact_u.shape)

In [ ]:
X_exact, T_exact = np.meshgrid(x_exact,t_exact)
print('X_exact:',X_exact.shape)
print('T_exact:',T_exact.shape)

X_exact_flatten = X_exact.flatten()[:,None]
T_exact_flatten = T_exact.flatten()[:,None]
data_star = np.hstack((X_exact_flatten,T_exact_flatten))
u_star = Exact_u.flatten()[:,None] 

In [ ]:
data_star.shape

In [ ]:
X_exact

In [ ]:
T_exact

In [ ]:
plt.imshow(Exact_u,extent=[-1, 1, 0, 1],
            origin='lower', cmap='rainbow')

plt.colorbar(shrink=.5)

plt.show()

In [ ]:
number_observe_data = 100

index_observe = (lhs(1,number_observe_data))*data_star.shape[0]
index_observe = np.floor(index_observe).reshape(number_observe_data,).astype(int)
observe_data = data_star[index_observe,:]
observe_u = u_star[index_observe,:]

observe_data = numpy_to_tensor(observe_data, var_name="observe_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
observe_u = numpy_to_tensor(observe_u, var_name="observe_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
print('J:',len(observe_u))

observe_data_x_inside = observe_data[:,0:1]
observe_data_t_inside = observe_data[:,1:2]

In [ ]:
n_test_data = 10000

index_observe = (lhs(1,n_test_data))*data_star.shape[0]
index_observe = np.floor(index_observe).reshape(n_test_data,).astype(int)
test_data = data_star[index_observe,:]
test_u = u_star[index_observe,:]

test_data = numpy_to_tensor(test_data, var_name="observe_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
test_u = numpy_to_tensor(test_u, var_name="observe_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

test_data_x_inside = observe_data[:,0:1]
test_data_t_inside = observe_data[:,1:2]

In [36]:
# C1 = torch.tensor(0.5, requires_grad=True)

In [ ]:
print('J:',len(observe_u))

In [38]:
def output_transform(data_input, u_input):
    
    x_in = data_input[:,0:1]
    t_in = data_input[:,1:2]
    
    return x_in**2 * torch.cos(torch.tensor(np.pi) * x_in) + t_in * (1 - x_in**2) * u_input

## Loss_f

In [39]:
def get_loss_f(x_grad, y_grad, PINNs, C, return_sequence='not'):

    E_inside = PINNs(torch.cat((x_grad,y_grad),axis=1))
    E_inside = output_transform(torch.cat((x_grad,y_grad),axis=1),E_inside)
    
    E_x = compute_higher_order_derivatives(E_inside, [x_grad])
    E_xx = compute_higher_order_derivatives(E_x, [x_grad])
    E_xxx = compute_higher_order_derivatives(E_xx, [x_grad])
    E_xxt = compute_higher_order_derivatives(E_xx, [y_grad])
    E_t = compute_higher_order_derivatives(E_inside, [y_grad])
    E_tx = compute_higher_order_derivatives(E_t, [x_grad])
    E_tt = compute_higher_order_derivatives(E_t, [y_grad])

    loss_term = E_t - C * E_xx - 5 * (E_inside - E_inside**3)

    if return_sequence == 'yes':
        return torch.square(loss_term)
    else:
        return torch.mean(torch.square(loss_term))

## loss_bound

In [40]:
def get_loss_bound(bound_x, bound_t, PINNs, C, return_sequence='not'):

    E_bound_x_zero = PINNs(torch.cat((bound_x,torch.zeros_like(bound_x)),axis=1)) 
    Exact_x_zero = bound_x**2*torch.cos(torch.tensor(np.pi)*bound_x)
    loss_bound_for_a = torch.mean(torch.square(E_bound_x_zero-Exact_x_zero))
    
    E_bound_fu_1_t = PINNs(torch.cat((-torch.ones_like(bound_t),bound_t),axis=1)) 
    Exact_fu_1_t = -torch.ones_like(E_bound_fu_1_t)
    loss_bound_for_b = torch.mean(torch.square(E_bound_fu_1_t-Exact_fu_1_t))
    
    E_bound_1_t = PINNs(torch.cat((torch.ones_like(bound_t),bound_t),axis=1))
    Exact_1_t = -torch.ones_like(E_bound_1_t)
    loss_bound_for_c = torch.mean(torch.square(E_bound_1_t-Exact_1_t))
    
    loss_bound_value = loss_bound_for_a+loss_bound_for_b+loss_bound_for_c
    
    return loss_bound_value

# PINN

In [41]:
#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [42]:
class hidden_layers(nn.Module):
    def __init__(self,input_number,output_number):
        super(hidden_layers, self).__init__()
        self.layer = nn.Linear(input_number,output_number)
    def forward(self, x):
        x = self.layer(x)
        x = torch.tanh(x)
        return x

class NN_H2 (nn.Module):
    def __init__(self,in_N, width, depth, out_N):

        super(NN_H2, self).__init__()
        self.in_N = in_N
        self.width = width
        self.depth = depth
        self.out_N = out_N

        self.stack = nn.ModuleList()

        self.stack.append(hidden_layers(in_N, width))

        for i in range(depth):
            self.stack.append(hidden_layers(width, width))

        self.stack.append(nn.Linear(width, out_N))
        
        
    def forward(self, x):
        for m in self.stack:
            x = m(x)
        return x

In [43]:
PINNs1 = NN_H2(2, 64, 4, 1)
PINNs1.cuda()

#PINNs1.apply(weights_init)
import torch.nn.init as init
for name, param in PINNs1.named_parameters():
    if 'weight' in name:
        init.xavier_uniform_(param)
        
optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

C1 = torch.tensor(0.1, requires_grad=True)
optimizer1.add_param_group({'params': [C1], 'lr': 0.001})

In [ ]:
############## Record list ###############
loss_all_1 = []
loss_f_1 = []
loss_f_for_collocation_1 = []
loss_f_for_T_1 = []
loss_f_excapt_T_1 = []
loss_T_1 = []
loss_T_clear_1 = []
loss_T_1_test_data = []
test_loss_1 = []
C1_list = []
############## Record list ###############


nIter1 = 10000
it = 0

while it<nIter1:
    #########loss f#########
    loss_f = get_loss_f(x_inside,t_inside,PINNs1,C1,return_sequence='not')
    
    #########loss T observation#########        
    E_observation = PINNs1(observe_data) 
    E_observation = output_transform(observe_data,E_observation)
    loss_observation = torch.mean(torch.square(E_observation-observe_u))              

    #########loss PI#########
    loss = loss_f+loss_observation
    
    #########  test_loss NRMSE  #########
    pre_u = PINNs1(test_data)
    pre_u = output_transform(test_data, pre_u) 
    test_loss = relative_l2_torch(pre_u,test_u)
    #########  test_loss NRMSE  #########
    
    ##############Record###############
    test_loss_1.append(test_loss)
    loss_f_1.append(loss_f.item())
    C1_list.append(C1.item())   
    loss_T_1.append(loss_observation.item()) 
    loss_all_1.append(loss.item())
    ##############Record###############

    if it % 1000 == 0:
        print('It:', it, 'train_loss:', loss.item(), 'test_loss:', test_loss)
        print(C1)
        
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    it = it + 1   
    
print('Final:', 'train_loss:', loss.item(), 'test_loss:', test_loss)